# *``lca_graphic``* : **A toolbox for daily LCA users**

---

**Purpose = to give understandable graphs for the daily LCA users to quickly analyze his LCA**
<br>
---

**Author** : Teo Lavisse, teo.lavisse@cea.fr       


## Initializing

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json
import os
import matrix_utils as mu
import bw_processing as bp
import math
import warnings
import ipywidgets as widgets
from ipywidgets import interact
import bw2analyzer as ba
import seaborn as sns
import textwrap

from utils import *
from dashboards import *

In [2]:
bd.projects.set_current("Visualization")

### Importing Databases

In [3]:
bd.databases


Databases dictionary with 3 object(s):
	US EEIO 1.1
	biosphere3
	ecoinvent 3.9_cutoff

In [4]:
db_USLCI=bd.Database("US EEIO 1.1")
db_EI=bd.Database("ecoinvent 3.9_cutoff")

###  Get activities and methods

#### US LCI

Get 2 activities and methods from the database

In [5]:
window_metal=next(node for node in db_USLCI if node['type'] == 'product' if node['name']=="Metal windows, doors, and architectural products; at manufacturer")
window_metal

'Metal windows, doors, and architectural products; at manufacturer' (, United States, ('31-33: Manufacturing', '3323: Architectural and Structural Metals Manufacturing'))

In [6]:
window_wood=next(node for node in db_USLCI if node['type'] == 'product' if node['name']=="Wooden windows, door, and flooring; at manufacturer")
window_wood

'Wooden windows, door, and flooring; at manufacturer' (, United States, ('31-33: Manufacturing', '3219: Other Wood Product Manufacturing'))

In [7]:
fu={window_metal:1,window_wood:1}

In [8]:
methods = [
    ('Impact Potential', 'GCC'),
    ('Impact Potential', 'HTOX'),
    ('Resource Use', 'LAND'),
    ('Resource Use', 'WATR'),
]

#### EI

In [9]:
act1 = db_EI.random()
act2 = db_EI.random()

fu = {act1: 1, act2: 1}

In [10]:
CC = [method for method in bd.methods if "IPCC 2021" in str(method) and "GWP100" in str(method)][0]
ADP = [method for method in bd.methods if "ADP" in str(method) if "ultimate reserves" in str(method) if 'EF v3.1 no LT' in str(method)][0]
TOX = [method for method in bd.methods if "human toxicity: carcinogenic no LT" in str(method) if 'EF v3.1' in str(method)][0]

impact_categories = [CC, ADP, TOX]

In [11]:
fu

{'market for maize silage' (kilogram, BR, None): 1,
 'electricity production, hydro, run-of-river' (kilowatt hour, ZA, None): 1}

##  Perform basic lca computations

In [12]:
fu

{'market for maize silage' (kilogram, BR, None): 1,
 'electricity production, hydro, run-of-river' (kilowatt hour, ZA, None): 1}

In [13]:
df=lca_comparison(fu, impact_categories)
df

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)","(EF v3.1 no LT, material resources: metals/minerals no LT, abiotic depletion potential (ADP): elements (ultimate reserves) no LT)","(EF v3.1 no LT, human toxicity: carcinogenic no LT, comparative toxic unit for human (CTUh) no LT)"
market for maize silage,0.061560,4.588674e-07,9.477998e-11
"electricity production, hydro, run-of-river",0.004524,2.349866e-08,1.365371e-11


In [14]:
df_norm=df.T.apply(lambda x: x/x.max(), axis=1)*100 #to normalize the results for each impact category
df_norm

,market for maize silage,"electricity production, hydro, run-of-river"
"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)",100.0,7.349681
"(EF v3.1 no LT, material resources: metals/minerals no LT, abiotic depletion potential (ADP): elements (ultimate reserves) no LT)",100.0,5.121014
"(EF v3.1 no LT, human toxicity: carcinogenic no LT, comparative toxic unit for human (CTUh) no LT)",100.0,14.405686


## Perform a basic contribution analysis

In [15]:
act_topscore(fu, CC)

'market for maize silage' (kilogram, BR, None)

In [36]:
df=contributions_df(act1,CC,limit=5,limit_type='number',norm=True, group_by_other=True)

In [37]:
df

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)"
0f42c1a81749fe6c6ea9b8d5876e1921,-6.596539
a819bfd13adefc9d2f6c05444dc985d0,4.371495
e23afab9f29944ff46bf1e1c02a90e3e,4.600684
7f4f4179f172f5b9cf7e1d923b6b1897,5.354151
Others,45.720657
c36959056f9a688a2bdc608ba7d4f8cb,46.549551


In [38]:
df.values.sum()

100.0

## Display all thes informations in a nice dashboard

### Defining colors

In [39]:
#define standard color palette:
colors = ["#F08C2E", "#7f6000", "#72AF42", "#A32683"]

#create longer color list for complex figures
color_div_YlBr=sns.color_palette('YlOrBr',6)
color_seq_green=sns.color_palette('Greens',6)
color_seq_RdPu=sns.color_palette('RdPu',6)
color_seq_org=sns.color_palette('Oranges',5)

colors.extend(colors)
# colors.extend(color_div_YlBr)
# colors.extend(color_seq_green)
# colors.extend(color_seq_RdPu)
# colors.extend(color_seq_org)

df_color=pd.DataFrame(index=impact_categories,data=[colors[c] for c in range(len(impact_categories))]).T

### Dashboard compare

In [40]:
compare(fu, impact_categories, sharex=False, cols=2, func_unit="kg")

### Dashboard hotspots

In [21]:
df = lca_comparison(fu, impact_categories)
df

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)","(EF v3.1 no LT, material resources: metals/minerals no LT, abiotic depletion potential (ADP): elements (ultimate reserves) no LT)","(EF v3.1 no LT, human toxicity: carcinogenic no LT, comparative toxic unit for human (CTUh) no LT)"
market for maize silage,0.061560,4.588674e-07,9.477998e-11
"electricity production, hydro, run-of-river",0.004524,2.349866e-08,1.365371e-11


In [22]:
act1

'market for maize silage' (kilogram, BR, None)

In [23]:
df_contrib = contributions_df(act1,CC,limit=0.05)
df_contrib

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)"
0f42c1a81749fe6c6ea9b8d5876e1921,-0.004061
c36959056f9a688a2bdc608ba7d4f8cb,0.028656


In [24]:
hotspots(fu, impact_categories, limit=0.05)

### Dashboard impact_transfer

In [25]:
impact_transfer({act1:1,act2:1}, impact_categories,limit=5, cols=2, func_unit="kWh")

## Generic function : lca_graphic

In [27]:
lca_graphic(fu,impact_categories)

In [26]:
lca_graphic({window_metal:1,window_wood:1},methods, reference_category=('Impact Potential', 'GCC'), func_unit="kg")